tsfresh提取特征

In [1]:
import joblib
import pandas as pd
import numpy as np
from tsfresh import select_features
from tsfresh import extract_features
# from tsfresh.feature_extraction import EfficientFCParameters
# from tsfresh.feature_extraction import ComprehensiveFCParameters
from joblib import Parallel, delayed
import os

In [2]:
minimal_parameters = {
     'length': None,
     'abs_energy': None,
    
    'mean':None,
    'mean_abs_change': None,
    'mean_change': None,
     'variance': None,
     'skewness': None,
     'kurtosis': None,

}

# fc_parameters = {
#     'length': None,
#     'abs_energy': None,
    
#      'median': None,
#      'mean': None,
#      'standard_deviation': None,
#      'variance': None,
#      'skewness': None,
#      'kurtosis': None,

#      'sample_entropy': None,

#      'ar_coefficient': [{'coeff': 0, 'k': 10},
#                           {'coeff': 1, 'k': 10},
#                           {'coeff': 2, 'k': 10},
#                           {'coeff': 3, 'k': 10},
#                           {'coeff': 4, 'k': 10}],
   
#     'fft_aggregated': [{'aggtype': 'centroid'},
#                           {'aggtype': 'variance'},
#                           {'aggtype': 'skew'},
#                           {'aggtype': 'kurtosis'}],

#     'agg_autocorrelation': [{'f_agg': 'mean'},
#                             {'f_agg': 'median'},
#                             {'f_agg': 'var'}],

#     'number_cwt_peaks': [{'n': 1}, {'n': 5}],
#     'binned_entropy': [{'max_bins': 10}],
#     'approximate_entropy': [{'m': 2, 'r': 0.1},
#     {'m': 2, 'r': 0.3},
#     {'m': 2, 'r': 0.5},
#     {'m': 2, 'r': 0.7},
#     {'m': 2, 'r': 0.9}],
#     'linear_trend': [{'attr': 'pvalue'},
#     {'attr': 'rvalue'},
#     {'attr': 'intercept'},
#     {'attr': 'slope'},
#     {'attr': 'stderr'}],
# }
settings = minimal_parameters


In [3]:
def gen_tsfresh_feature_basis(sensor,settings):
    '''
    描述：
        settings，使用tsfresh提取指定特征
    参数：
        sensor：
        settings(dict): 想要提取的特征
    '''

    extracted_features = extract_features(sensor, column_id='id', column_sort='sort_col', n_jobs=48, 
                                          default_fc_parameters=settings, disable_progressbar = False)
    return extracted_features


In [4]:
def gen_tsfresh_feature_parallel(data_no, csv_nos, settings):
    '''
    描述：
        并行提取多个senosr文件的tsfresh特征
    参数：
        data_no：第几个plc
        csv_nos：plc对应的sensor文件个数
    '''
    
    input_dir = './sensors_ad/%s/'%data_no
    output_dir = './sensors_tsfresh_minimal/%s/'%data_no
    
    if not os.path.exists('./sensors_tsfresh_minimal/'):
        os.mkdir('./sensors_tsfresh_minimal')
    if not os.path.exists('./sensors_tsfresh_minimal/%s'%data_no):
        os.mkdir('./sensors_tsfresh_minimal/%s'%data_no)
    
    def basis_func(idx):
        sensor = joblib.load(input_dir + '%d.lz4'%idx)
        tmp = gen_tsfresh_feature_basis(sensor, settings)
        joblib.dump(tmp, output_dir+'%d.lz4'%idx, compress='lz4')
        
    Parallel(n_jobs=10,verbose=1)(delayed(basis_func)(i) for i in range(1,csv_nos+1))

In [5]:
gen_tsfresh_feature_parallel('01', 48, settings)

[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  48 out of  48 | elapsed:  2.4min finished


In [6]:
gen_tsfresh_feature_parallel('02', 48 , settings)

[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  48 out of  48 | elapsed:  2.4min finished


In [7]:
gen_tsfresh_feature_parallel('03', 37, settings)

[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  37 out of  37 | elapsed:  1.9min finished


In [2]:
# import joblib
# test = joblib.load('./sensors_tsfresh_minimal/01/1.lz4')
# test.shape